# LogisticRegression avec recherche des hyperparamètres optimaux (GridSearch), Cross Validation et Random Oversampling.

## 1ère partie : Régression Logistique sans rééchantillonage

##  Import des bibliothèques nécessaires

In [2]:
import pandas as pd;
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from imblearn.metrics import geometric_mean_score 
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pip install missingno

Note: you may need to restart the kernel to use updated packages.


## import missingno as msno

In [4]:
"""
L'utilisation du dtype "catégorie" est un excellent moyen d'économiser de la mémoire et d'améliorer les performances. 
La spécification des dtypes réduira la quantité de mémoire nécessaire au chargement de l'ensemble de données. 
C'est très utile lorsque l'on a de grands ensembles de données.

Création d'un dictionnaire avec les noms des colonnes comme clés et "catégorie" comme valeurs

"""

df_dtypes = {
  "catr"       :"category", 
  "circ"       :"category", 
  "prof"       :"category",
  "plan"       :"category",
  "surf"       :"category",
  "situ"       :"category",
  "lum"        :"category",
  "agg"        :"category",
  "int"        :"category",
  "atm"        :"category",
  "col"        :"category",
  "infra"      :"category",
  "obs"        :"category",
  "obsm"       :"category",
  "choc"       :"category",
  "manv"       :"category",
  "num_veh"    :"category",
  "catv_Label" :"category",
  "catu"       :"category",
  "sexe"       :"category",
  "trajet"     :"category",
  "secuUn"     :"category",
  "secuDeux"   :"category",
  "tranches_ages" :"category"
  }
 

## Read in the CSV using the dtypes parameter

In [5]:
df = pd.read_csv('../data/fusion_def_clean.csv', dtype=df_dtypes, low_memory=False)
df.head(10)

,Unnamed: 0,num_acc,mois,jour,lum,agg,int,atm,col,com,...,actp,etatp,an_nais,secuUn,secuDeux,an_naiss,age_acc,age_acc_seconds,age_acc_an,tranches_ages
0,0,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,...,0,Non renseigné,1976-01-01,Ceinture,Ceinture,1976-01-01,10593 days,9.152352e+08,29.021918,23-31
1,1,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,...,0,Non renseigné,1968-01-01,Ceinture,Ceinture,1968-01-01,13515 days,1.167696e+09,37.027397,32-40
2,2,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,...,0,Non renseigné,1964-01-01,Ceinture,Ceinture,1964-01-01,14976 days,1.293926e+09,41.030137,41-50
3,3,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,...,0,Non renseigné,2004-01-01,Dispositif enfants,Ceinture,2004-01-01,366 days,3.162240e+07,1.002740,0-11
4,4,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,...,0,Non renseigné,1998-01-01,Ceinture,Ceinture,1998-01-01,2557 days,2.209248e+08,7.005479,0-11
5,5,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,...,0,Non renseigné,1991-01-01,Ceinture,Ceinture,1991-01-01,5114 days,4.418496e+08,14.010959,12-22
6,6,200500000002,1,21,Plein jour,En agglomération,Hors intersection,Normale,Deux véhicules - frontale,51,...,0,Non renseigné,1955-01-01,Ceinture,Ceinture,1955-01-01,18263 days,1.577923e+09,50.035616,50+
7,7,200500000002,1,21,Plein jour,En agglomération,Hors intersection,Normale,Deux véhicules - frontale,51,...,0,Non renseigné,1979-01-01,Casque,Ceinture,1979-01-01,9497 days,8.205408e+08,26.019178,23-31
8,8,200500000003,1,21,Nuit sans éclairage public,Hors agglomération,Hors intersection,Pluie légère,Deux véhicules - frontale,51,...,0,Non renseigné,1983-01-01,Casque,Ceinture,1983-01-01,8036 days,6.943104e+08,22.016438,12-22
9,9,200500000003,1,21,Nuit sans éclairage public,Hors agglomération,Hors intersection,Pluie légère,Deux véhicules - frontale,51,...,0,Non renseigné,1956-01-01,Casque,Ceinture,1956-01-01,17898 days,1.546387e+09,49.035616,41-50


## Nombre de lignes et de colonnes dans le dataframe

In [6]:
df.shape

(2192779, 48)

## Affichage des informations du DataFrame

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192779 entries, 0 to 2192778
Data columns (total 48 columns):
 #   Column              Dtype   
---  ------              -----   
 0   Unnamed: 0          int64   
 1   num_acc             int64   
 2   mois                int64   
 3   jour                int64   
 4   lum                 category
 5   agg                 category
 6   int                 category
 7   atm                 category
 8   col                 category
 9   com                 object  
 10  dep                 object  
 11  annee_x_x           int64   
 12  date                object  
 13  jour_de_la_semaine  int64   
 14  heure               int64   
 15  catr                category
 16  circ                category
 17  nbv                 float64 
 18  vosp                object  
 19  prof                category
 20  plan                category
 21  surf                category
 22  infra               category
 23  situ                category
 24

## Préparation des données - Suppression des colonnes inutiles  

In [8]:
columns_to_drop = ['Unnamed: 0', 'senc', 'locp', 'actp', 'etatp', 'an_nais', 'an_naiss', 'age_acc', 'age_acc_seconds',
                   'age_acc_an', "permis"]

df1 = df.drop(columns=columns_to_drop, axis=1)
df1.head(10)

,num_acc,mois,jour,lum,agg,int,atm,col,com,dep,...,num_veh,catv_Label,place,catu,grav,sexe,trajet,secuUn,secuDeux,tranches_ages
0,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,59,...,A01,Voitures légères (VL) seule,1.0,Conducteur,Blessé léger,Masculin,Domicile – travail,Ceinture,Ceinture,23-31
1,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,59,...,B02,Voitures légères (VL) seule,1.0,Conducteur,Blessé hospitalisé,Féminin,Courses – achats,Ceinture,Ceinture,32-40
2,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,59,...,B02,Voitures légères (VL) seule,2.0,Passager,Indemne,Masculin,Non renseigné,Ceinture,Ceinture,41-50
3,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,59,...,B02,Voitures légères (VL) seule,4.0,Passager,Indemne,Masculin,Non renseigné,Dispositif enfants,Ceinture,0-11
4,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,59,...,B02,Voitures légères (VL) seule,5.0,Passager,Indemne,Masculin,Non renseigné,Ceinture,Ceinture,0-11
5,200500000001,1,12,Nuit sans éclairage public,En agglomération,Hors intersection,Normale,Deux véhicules – par le coté,11,59,...,B02,Voitures légères (VL) seule,3.0,Passager,Indemne,Féminin,Non renseigné,Ceinture,Ceinture,12-22
6,200500000002,1,21,Plein jour,En agglomération,Hors intersection,Normale,Deux véhicules - frontale,51,59,...,A01,Voitures légères (VL) seule,1.0,Conducteur,Indemne,Masculin,Promenade – loisirs,Ceinture,Ceinture,50+
7,200500000002,1,21,Plein jour,En agglomération,Hors intersection,Normale,Deux véhicules - frontale,51,59,...,B02,Deux ou trois roues de petite cylindrée (<= 50...,1.0,Conducteur,Blessé hospitalisé,Masculin,Promenade – loisirs,Casque,Ceinture,23-31
8,200500000003,1,21,Nuit sans éclairage public,Hors agglomération,Hors intersection,Pluie légère,Deux véhicules - frontale,51,59,...,A01,Deux ou trois roues de petite cylindrée (<= 50...,1.0,Conducteur,Indemne,Masculin,Domicile – travail,Casque,Ceinture,12-22
9,200500000003,1,21,Nuit sans éclairage public,Hors agglomération,Hors intersection,Pluie légère,Deux véhicules - frontale,51,59,...,B02,Deux ou trois roues de petite cylindrée (<= 50...,1.0,Conducteur,Blessé hospitalisé,Masculin,Domicile – travail,Casque,Ceinture,41-50


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192779 entries, 0 to 2192778
Data columns (total 37 columns):
 #   Column              Dtype   
---  ------              -----   
 0   num_acc             int64   
 1   mois                int64   
 2   jour                int64   
 3   lum                 category
 4   agg                 category
 5   int                 category
 6   atm                 category
 7   col                 category
 8   com                 object  
 9   dep                 object  
 10  annee_x_x           int64   
 11  date                object  
 12  jour_de_la_semaine  int64   
 13  heure               int64   
 14  catr                category
 15  circ                category
 16  nbv                 float64 
 17  vosp                object  
 18  prof                category
 19  plan                category
 20  surf                category
 21  infra               category
 22  situ                category
 23  obs                 category
 24

## Transformation des variables catégorielles (qualitatives) en variables   quantitatives afin qu'elles puissent être interprétées par un modèle de machine learning

## Traitement des variables catégorielles nominales : One Hot Encoding (dichotomisation) sur les variables suivantes : 
'agg', 'int', 'atm', 'col', 'catr', 'circ', 'vosp', 'prof', 'plan', 'surf', 'infra', 'situ', 'obs', 'obsm',
'choc', 'manv', 'catv_Label', 'catu', 'sexe', 'trajet', 'secuUn', 'secuDeux'

In [10]:
vosp = df1['vosp']
df1 = df1.join(pd.get_dummies(vosp, prefix='vosp'))

In [11]:
agg = df1['agg']
df1 = df1.join(pd.get_dummies(agg, prefix='agg'))

In [12]:
int = df1['int']
df1 = df1.join(pd.get_dummies(int, prefix='int'))

In [13]:
atm = df1['atm'] 
df1 = df1.join(pd.get_dummies(atm, prefix='atm'))

In [14]:
col = df1['col'] 
df1 = df1.join(pd.get_dummies(col, prefix='col'))

In [15]:
catr = df1['catr'] 
df1 = df1.join(pd.get_dummies(catr, prefix='catr'))

In [16]:
circ = df1['circ'] 
prof = df1['prof'] 
plan = df1['plan'] 

df1 = df1.join(pd.get_dummies(circ, prefix='circ'))
df1 = df1.join(pd.get_dummies(prof, prefix='prof'))
df1 = df1.join(pd.get_dummies(plan, prefix='plan'))

In [17]:
surf = df1['surf'] 
infra = df1['infra'] 
situ = df1['situ'] 

df1 = df1.join(pd.get_dummies(surf, prefix='surf'))
df1 = df1.join(pd.get_dummies(infra, prefix='infra'))
df1 = df1.join(pd.get_dummies(situ, prefix='situ'))

In [18]:
obs = df1['obs'] 
obsm = df1['obsm'] 
choc = df1['choc']

df1 = df1.join(pd.get_dummies(obs, prefix='obs'))
df1 = df1.join(pd.get_dummies(obsm, prefix='obsm'))
df1 = df1.join(pd.get_dummies(choc, prefix='choc'))

In [19]:
catu = df1['catu'] 
sexe = df1['sexe'] 
trajet = df1['trajet'] 

df1 = df1.join(pd.get_dummies(catu, prefix='catu'))
df1 = df1.join(pd.get_dummies(sexe, prefix='sexe'))
df1 = df1.join(pd.get_dummies(trajet, prefix='trajet'))

In [20]:
secuUn = df1['secuUn'] 
secuDeux = df1['secuDeux'] 

df1 = df1.join(pd.get_dummies(secuUn, prefix='secuUn'))
df1 = df1.join(pd.get_dummies(secuDeux, prefix='secuDeux'))

In [21]:
catv_Label = df1['catv_Label'] 
df1 = df1.join(pd.get_dummies(catv_Label, prefix='catv'))

In [22]:
manv = df1['manv'] 
df1 = df1.join(pd.get_dummies(manv, prefix='manv'))

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192779 entries, 0 to 2192778
Columns: 244 entries, num_acc to manv_Traversant la chaussée
dtypes: category(24), float64(2), int64(6), object(5), uint8(207)
memory usage: 702.7+ MB


## Conservation des variables indicatrices correspondants aux différentes modalités des variables qualitatives dichotomisées et suppression des variables qualitatives concernées

In [24]:
df1 = df1.drop(['agg', 'int', 'atm', 'col', 'catr', 'circ', 'vosp', 'prof', 'plan', 'surf', 'infra', 'situ', 'obs', 'obsm', 
                'choc', 'manv', 'catv_Label', 'catu', 'sexe', 'trajet', 'secuUn', 'secuDeux'], axis=1)

## Suppression de la variable date qui n'apporte aucune information supplémentaire par rapport aux variables 'jour', 'mois' et 'annee_x_x'

In [25]:
df1 = df1.drop('date', axis=1)

In [26]:
df1.head()

,num_acc,mois,jour,lum,com,dep,annee_x_x,jour_de_la_semaine,heure,nbv,...,manv_Inconnue,manv_Manœuvre de stationnement,manv_Manœuvre d’évitement,"manv_Même sens, même file",manv_Non renseigné,manv_Ouverture de porte,manv_Sans changement de direction,manv_Tournant - A droite,manv_Tournant - A gauche,manv_Traversant la chaussée
0,200500000001,1,12,Nuit sans éclairage public,11,59,2005,2,19,2.0,...,0,0,0,0,0,0,1,0,0,0
1,200500000001,1,12,Nuit sans éclairage public,11,59,2005,2,19,2.0,...,0,0,0,0,0,0,0,0,0,0
2,200500000001,1,12,Nuit sans éclairage public,11,59,2005,2,19,2.0,...,0,0,0,0,0,0,0,0,0,0
3,200500000001,1,12,Nuit sans éclairage public,11,59,2005,2,19,2.0,...,0,0,0,0,0,0,0,0,0,0
4,200500000001,1,12,Nuit sans éclairage public,11,59,2005,2,19,2.0,...,0,0,0,0,0,0,0,0,0,0


## Conversion des variables catégorielles ordinales et de la variable cible en variables numériques

In [27]:
from sklearn.preprocessing import LabelEncoder

In [28]:
le = LabelEncoder()
df1['lum'] = le.fit_transform(df['lum'])
df1['num_veh'] = le.fit_transform(df['num_veh'])
df1['place'] = le.fit_transform(df['place'])
df1['tranches_ages'] = le.fit_transform(df['tranches_ages'])
df1['grav'] = le.fit_transform(df['grav'])

In [29]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192779 entries, 0 to 2192778
Columns: 221 entries, num_acc to manv_Traversant la chaussée
dtypes: float64(1), int32(4), int64(7), object(2), uint8(207)
memory usage: 633.6+ MB


In [30]:
for col in df1.columns:
    print(col, df1[col].dtype)    

num_acc int64
mois int64
jour int64
lum int32
com object
dep object
annee_x_x int64
jour_de_la_semaine int64
heure int64
nbv float64
num_veh int32
place int64
grav int32
tranches_ages int32
vosp_Bande cyclable(2) uint8
vosp_Non renseigné(-1) uint8
vosp_Piste cyclable(1) uint8
vosp_Sans objet(0) uint8
vosp_Voie réservée(3) uint8
agg_En agglomération uint8
agg_Hors agglomération uint8
int_Autre intersection uint8
int_Giratoire uint8
int_Hors intersection uint8
int_Intersection en T uint8
int_Intersection en X uint8
int_Intersection en Y uint8
int_Intersection à plus de 4 branches uint8
int_Passage à niveau uint8
int_Place uint8
atm_Autre uint8
atm_Brouillard - fumée uint8
atm_Neige - grêle uint8
atm_Non renseigné uint8
atm_Normale uint8
atm_Pluie forte uint8
atm_Pluie légère uint8
atm_Temps couvert uint8
atm_Temps éblouissant uint8
atm_Vent fort - tempête uint8
col_Autre collision uint8
col_Deux véhicules - frontale uint8
col_Deux véhicules – par le coté uint8
col_Deux véhicules – par l’

## Conversion des variables 'com' et 'dep' (object/string) en float

In [31]:
df1['com'] = df1['com'].str.replace(r'(2A)', '200', regex=True).replace(r'(2B)', '201', regex=True).astype('float')

In [32]:
df1['com'].dtype

dtype('float64')

In [33]:
df1['dep'] = df1['dep'].str.replace(r'(2A)', '200', regex=True).replace(r'(2B)', '201', regex=True).astype('float')

In [34]:
df1['dep'].dtype

dtype('float64')

## Affectation des "features" du dataset dans "data" et affectation de la variable cible dans "target" 

In [35]:
data = df1.drop('grav', axis=1)

In [36]:
target = df1['grav']

## Séparation des données en ensembles d'entraînement et de test

In [37]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

## Recherche des hyperparamètres optimaux avec GridSearch 

In [39]:
# Importation des packages
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [45]:
# Grid search cross validation

logreg=LogisticRegression() # Construisons un modèle de base
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]} # l1 lasso, l2 ridge
logreg_cv=GridSearchCV(logreg, grid, cv=10)
logreg_cv.fit(X_train,y_train)

print("tuned hyperparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

C:\Users\clevr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
70 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\clevr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\clevr\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\clevr\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Val

tuned hyperparameters :(best parameters)  {'C': 0.001, 'penalty': 'l2'}
accuracy : 0.4088767505618751


In [46]:
print("tuned hyperparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hyperparameters :(best parameters)  {'C': 0.001, 'penalty': 'l2'}
accuracy : 0.4088767505618751


In [ ]:
"""
Synthèse des résultats :
accuracy : 0.4088767505618751
tuned hyperparameters :(best parameters)  {'C': 0.001, 'penalty': 'l2'}
model = LogisticRegression(C=0.001, penalty="l2")

"""

## Entrainement du modèle de classification optimisé sur les données d'entraînement

In [63]:
logreg2=LogisticRegression(C=0.001, penalty="l2")

from time import time
t0 = time()
logreg2.fit(X_train, y_train)
t1 = time() - t0
print("Entrainement réalisé en {} secondes".format(round(t1,3)))

Entrainement réalisé en 61.586 secondes


## Évaluation du modèle de classification optimisé

In [64]:
# Affichage du score de prédiction sur l'ensemble d'entrainement et calcul du temps d'exécution

In [86]:
from sklearn.metrics import accuracy_score

In [85]:
t0 = time()

predict_train = logreg2.predict(X_train)
train_acc = accuracy_score(y_train, predict_train)
print('Score prédiction sur les données entrainement :', train_acc)

tt = time() - t0
print("Réalisé en {} secondes".format(round(tt,3)))


Score prédiction sur les données entrainement : 0.40887675056135964
Réalisé en 3.831 secondes


In [87]:
# Affichage du score de prédiction sur l'ensemble de test et calcul du temps d'exécution

In [88]:
t0 = time()

predict_test = logreg2.predict(X_test)
test_acc = accuracy_score(y_test, predict_test)
print('Score prédiction sur les données de test :',test_acc)

tt = time() - t0
print("Réalisé en {} secondes".format(round(tt,3)))

Score prédiction sur les données de test : 0.4101825080491431
Réalisé en 0.924 secondes


## Cross Validation avec le modèle de classification optimisé

In [52]:
from sklearn.model_selection import StratifiedKFold
cross_val = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

'\ncross_val = KFold(n_splits=5, shuffle=True, random_state=42)\n\n'

In [53]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [68]:
# Calcul du Mean AUC Score

In [56]:
from sklearn.model_selection import cross_val_score
logreg2 = LogisticRegression(C=0.001, penalty="l2")
scores = cross_val_score(logreg2, X_train, y_train, cv=cross_val, scoring='roc_auc_ovo')

In [58]:
print("Mean AUC Score - Logistic Regression: ", scores.mean())

Mean AUC Score - Logistic Regression:  0.5095779598449728


In [59]:
# Calcul du balanced_accuracy 

In [69]:
from sklearn.metrics import balanced_accuracy_score 
predict_test = logreg2.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_test, predict_test)
print(balanced_accuracy)

0.25


In [71]:
# Calcul du geometric mean score

In [72]:
from imblearn.metrics import geometric_mean_score
geo_mean_score = geometric_mean_score(y_test, predict_test)
print("Geometric Mean Score :", geo_mean_score)

Geometric Mean Score : 0.0


In [73]:
## Calcul du balanced accuracy score

In [74]:
balanced_accuracy_score = balanced_accuracy_score(y_test, predict_test)
print("Balanced Accuracy Score :", balanced_accuracy_score)

Balanced Accuracy Score : 0.25


In [75]:
## Calcul du score de performance sur l'ensemble d'apprentissage 

print("train score", logreg2.score(X_train, y_train))

train score 0.40887675056135964


In [76]:
## Calcul du score de performance sur l'ensemble de test

print("test score", logreg2.score(X_test, y_test))

test score 0.4101825080491431


## Calcul de la matrice de confusion - Méthode 1 : à l'aide de sklearn

In [77]:
cm = confusion_matrix(y_test, predict_test)
print(cm)

[[     0      0  89903      0]
 [     0      0 157066      0]
 [     0      0 179888      0]
 [     0      0  11699      0]]


## Calcul de la matrice de confusion - Méthode 2 : à l'aide de pandas

In [78]:
cm = pd.crosstab(y_test, predict_test, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm

Classe prédite,2
Classe réelle,
0,89903
1,157066
2,179888
3,11699


## Commentaires

In [ ]:
"""
La matrice de confusion nous montre que la totalité des usagers accidentés de l'ensemble de test ont été classés
comme appartenant à la classe 2 (blessé hospitalisé). Le taux de bonnes prédictions sur cette classe est de 41 %. 

Le déséquilibre des classes est toujours aussi flagrant. Notre modèle est trop entrainé sur la classe 2, 
il est par conséquent tout naturellement meilleur dans la reconnaissance de celle ci et est donc biaisé. 

Aucune amélioration par rapport à un modèle de régression logistique sans Grid Search ni Cross Validation.

"""

## Affichage du rapport de classification pour évaluer les performances par classe

In [79]:
print("Rapport de classification :\n", classification_report(y_test, predict_test))

C:\Users\clevr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Rapport de classification :
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     89903
           1       0.00      0.00      0.00    157066
           2       0.41      1.00      0.58    179888
           3       0.00      0.00      0.00     11699

    accuracy                           0.41    438556
   macro avg       0.10      0.25      0.15    438556
weighted avg       0.17      0.41      0.24    438556



C:\Users\clevr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\clevr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Commentaires 

In [ ]:
"""
Le rapport de classification est rigoureusement identique à ce que nous avions obtenu avec un modèle
de régression logistique sans Grid Search ni Cross Validation. Aucune amélioration pour le moment.

Appliquons maintenant une méthode de rééchantillonnage : choix de la méthode Oversampling qui permet d'augmenter
le nombre d'observations des classes minoritaires afin d'arriver à un ratio classes minoritaires / classe majoritaire
satisfaisant.

"""

# 2ème partie : Régression Logistique avec rééchantillonage

In [80]:
pip install -U imbalanced-learn

In [81]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from sklearn.metrics import f1_score

## Application de la méthode Random Oversampling

In [55]:
# Random Oversampling

rOs = RandomOverSampler()
X_ro, y_ro = rOs.fit_resample(X_train, y_train)
print('Classes échantillon oversampled :', dict(pd.Series(y_ro).value_counts()))

Classes échantillon oversampled : {1: 717261, 0: 717261, 2: 717261, 3: 717261}


## Entraînement du modèle sur l'ensemble issu du RandomOversampler

In [56]:
logreg2.fit(X_ro, y_ro)

LogisticRegression()

## Évaluation du modèle sur l'ensemble issu du RandomOversampler

In [57]:
# Affichage du score de prédiction sur l'ensemble d'entrainement et calcul du temps d'exécution

t0 = time()

predict_train = logreg2.predict(X_ro)
train_acc = accuracy_score(y_ro, predict_train)
print('Score prédiction sur les données entrainement :',train_acc)

tt = time() - t0
print("Réalisé en {} secondes".format(round(tt,3)))

Score prédiction sur les données entrainement : 0.25
Réalisé en 56.892 secondes


In [58]:
# Affichage du score de prédiction sur l'ensemble de test et calcul du temps d'exécution

t0 = time()

predict_test = logreg2.predict(X_test)
test_acc = accuracy_score(y_test, predict_test)
print('Score prédiction sur les données de test :',test_acc)

tt = time() - t0
print("Réalisé en {} secondes".format(round(tt,3)))

Score prédiction sur les données de test : 0.3581435438119647
Réalisé en 1.02 secondes


In [59]:
# Calcul du geometric mean score

geometric_mean = geometric_mean_score(y_test, predict_test)
print("Geometric Mean Score :", geometric_mean)

Geometric Mean Score : 0.0


In [60]:
## Calcul du balanced accuracy score

balanced_accuracy = balanced_accuracy_score(y_test, predict_test)
print("Balanced Accuracy Score :", balanced_accuracy)

Balanced Accuracy Score : 0.25


In [61]:
## Calcul du balanced accuracy score

balanced_accuracy = balanced_accuracy_score(y_test, predict_test)
print("Balanced Accuracy Score :", balanced_accuracy)

Balanced Accuracy Score : 0.25


In [62]:
## Calcul du score sur l'ensemble d'apprentissage 

logreg2.score(X_train, y_train)

0.3581682602496946

In [63]:
## Calcul du score sur l'ensemble de test

logreg2.score(X_test, y_test)

0.3581435438119647

## Affichage de la matrice de confusion obtenue grâce aux prédictions sur l'ensemble de test

In [64]:
pd.crosstab(y_test, predict_test, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,1
Classe réelle,
0,89903
1,157066
2,179888
3,11699


## Commentaires

In [65]:
"""
La matrice de confusion nous montre que la totalité des usagers accidentés de l'ensemble de test ont été classés
comme appartenant à la classe 1 (blessé léger). Le taux de bonnes prédictions sur cette classe est de 35,81 %. 

Le déséquilibre des classes est toujours aussi flagrant. Notre modèle est trop entrainé sur la classe 1, 
il est par conséquent tout naturellement meilleur dans la reconnaissance de celle ci et est donc biaisé. 

Il n'y a aucune amélioration par rapport à un modèle de régression logistique sans Grid Search, ni Cross Validation
et sans rééchantillonage. Le taux de bonnes prédictions sur cette classe 1 (35.81 %) est rigoureusement identique 
au taux de bonnes prédictions obtenu sur la classe 2 (35.81 %) du modèle précédent avec Random Oversampling. 

Encore une fois, notre modèle est aussi performant qu'un modèle purement aléatoire qui attribuerait une classe
en la choisissant au hasard uniformément.

"""

"\nLa matrice de confusion nous montre que la totalité des usagers accidentés de l'ensemble de test ont été classés\ncomme appartenant à la classe 1 (blessé léger) alors que cette classe ne concerne en réalité que 35.81 % des accidents.\n\nPar conséquent, notre modèle est encore aussi performant qu'un modèle purement aléatoire qui attribuerait une classe\nen la choisissant au hasard uniformément.\n\n"

## Rapport de résultat sur l'ensemble de test obtenu par classification_report_imbalanced

In [66]:
print(classification_report_imbalanced(y_test, predict_test))

C:\Users\clevr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00     89903
          1       0.36      1.00      0.00      0.53      0.00      0.00    157066
          2       0.00      0.00      1.00      0.00      0.00      0.00    179888
          3       0.00      0.00      1.00      0.00      0.00      0.00     11699

avg / total       0.13      0.36      0.64      0.19      0.00      0.00    438556



## Commentaires 

In [ ]:
"""
La précision et le rappel sont nuls pour les classes 0, 2 et 3. 
Ces classes n'ont pas du tout été bien gérées par le modèle.

Concernant la classe 1 (blessé léger), la précision est de 36 %, toutefois le rappel est élevé (100 %).
Cette fois-ci, la classe 1 a bien été gérée par le modèle, celle-ci a bien été détectée.

Cependant, comme cela a été mentionné précedement, il faut relativiser ce résultat.
Concernant le F1-score, celui-ci est de 53 %, mais la performance de bonnes prédictions sur cette classe 
est toute relative compte tenu du déséquilibre flagrant avec les autres classes.

En conclusion, la méthode de rééchantillonage Random Oversampling n'a pas permis d'améliorer les performances
de notre modèle. La majorité des classes n'étant pas bien gérées et la problématique de déséquilibre de classes
est toujours bien présent.

"""

## Conclusion générale

In [ ]:
"""
Les résultats obtenus ne sont toujours pas satisfaisants.

Le Random Oversampling après l'étape Grid Search et CV n'a pas produit les résultats escomptés. 
Les performances de notre modèle Regression Logistic n'ont pas été améliorées. 
Notre modèle est aussi performant qu'un modèle purement aléatoire qui attribuerait une classe
en la choisissant au hasard uniformément. 

Par ailleurs, nous avons calculé la moyenne géometrique (geometric mean) qui s'avère utile
pour les problèmes de classification déséquilibrée : il s'agit de la racine du produit
de la sensibilité et de la spécificité. Celle-ci est nulle, par conséquent le modèle
n'est absolument pas acceptable pour notre problème.
Il peut s’agir de l’algorithme utilisé qui ne convient pas à nos données. 

Puis par la suite, nous allons expérimenter un modèle RandomForestClassifier qui sera entrainé afin de comparer
les performances obtenues avec le modèle Regression Logistic. Il s'avère que les modèles d’ensemble à base
d’arbres comme les RandomForest sont généralement mieux adaptés aux données déséquilibrées.

En parallèle, des modèles d’ensembles, comme le Boosting ou Bagging, seront entrainés par d'autres membres
de notre équipe. 
Il s'avère que ces modèles sont entraînés à chaque étape de l’algorithme sur un échantillon rééquilibré automatiquement
entre les différentes classes.

"""